### Implementation of calculation of links importance using other languages wiki articles together

In [1]:
from wikipydia import wikipedia, wikilinks, url
from wikipydia.url import QuotedURL, UnquotedURL

In [2]:
#for ll in wikipedia.get_article_langlinks(UnquotedURL("C++")):
    #print(ll[2].quoted)

In [3]:
from neo4j.v1 import GraphDatabase, basic_auth

#from urllib.parse import unquote

class Wiki4Neo():
    def __init__(self, user="neo4j", password="lucas"):
        """Connect to neo4j db"""
        self.driver = GraphDatabase.driver("bolt://localhost:7687", auth=basic_auth(user, password))
        
        #session = driver.session()
        #session.run("CREATE (a:Person {name: {name}, title: {title}})",{"name": "Arthur", "title": "King"})
        #result = session.run("MATCH (a:Person) WHERE a.name = {name} RETURN a.name AS name, a.title AS title",{"name": "Arthur"})
        #for record in result:
        #    print("%s %s" % (record["title"], record["name"]))
        #session.close()

    def save_article(self, article, links, url, lang):
        #Ensure to escape quotes from the title and url before query
        #"Lu'cas".replace("'", "\\'")
        #pageInfo.title = pageInfo.title.replace(/['\\]/g, "\\$&");
        #pageInfo.url = pageInfo.url.replace(/['\\]/g, "\\$&");

        article_title = escp(article.title())
        url = escp(url)

        #Construct query
        neo_query = "\n".join([
            'MERGE (article:WikiArticle{{title:"{}", pageId:{}}})'.format(article_title, article.page_id()), 
            'ON CREATE SET article.lang = "{}"'.format(lang), 
            'MERGE (articleUrl:Wikiurl{{url_lang:"{}"}})'.format(url.unquoted + "_" + lang),
            'ON CREATE SET articleUrl.url = "{}", articleUrl.lang = "{}"'.format(url.unquoted, lang),
            'SET articleUrl.articleId = {}'.format(article.page_id()),
            "CREATE UNIQUE (articleUrl)-[:RedirectsTo]->(article)"
        ])
        
        #Create queries for links to the article
        for i, (link_href, link_score) in enumerate(links):
            #Ensure to escape quotes from the link before query
            #var link = pageInfo.links[i].replace(/['\\]/g, "\\$&");
            _link_href = escp(QuotedURL(link_href)) #Remove url quotes from the href

            neo_query += "\n\n" + "\n".join([
                'MERGE (articleLink{}:Wikiurl{{url_lang: "{}"}})'.format(i, _link_href.unquoted+"_"+lang),
                'ON CREATE SET articleLink{}.url = "{}", articleLink{}.lang = "{}"'.format(i,_link_href.unquoted,i,lang),
                'CREATE UNIQUE (article)-[:LinksTo{{score:{}}}]->(articleLink{})'.format(link_score, i)
            ])
            
        #Create query to create direct connection between articles
        if False:
            neo_query += "\n"

            #Add in/out ConnectsTo relation to this article 
            neo_query += "\n".join([
                "WITH article MATCH (article)-[:LinksTo]->(:Wikiurl)-[:RedirectsTo]->(targetArticle:Article)",
                "CREATE UNIQUE (article)-[:ConnectsTo]->(targetArticle)",
                "WITH article MATCH (targetArticle:Article)-[:LinksTo]->(:Wikiurl)-[:RedirectsTo]->(article)", 
                " CREATE UNIQUE (targetArticle)-[:ConnectsTo]->(article)"
            ])
            
        neo_query += "\nRETURN id(article) as article_id"

        session = self.driver.session()
        results = session.run(neo_query)
        session.close()
        
        for record in results:
            return record["article_id"]

In [4]:
def escp(text):
    return text
    """Function to escape special characters."""
    new_text = text
    new_text = new_text.replace('"', '\\"') #Two bars because one escape the python code and the other the query str
    new_text = new_text.replace("'", "\\'")
    
    return new_text
    
print(escp('test"'))
print(escp("test'"))

test"
test'


In [5]:
w4n = Wiki4Neo()

In [6]:
def get_or_download_pagedata_by_href(href, lang):
    """
    Get data from database, downloading missing data.
    """
    
    #start_time = time.time()
    
    #1. Get the wikiarticle data from the db by the href 
    neo_query = "\n".join([
        'MATCH (n:Wikiurl{{url:"{}", lang:"{}"}})-[:RedirectsTo]->(a:WikiArticle)',
        'RETURN a.title as title, a.pageId as pageid, id(a) as id'
    ]).format(escp(href.unquoted), lang)
    
    session = w4n.driver.session()
    results = session.run(neo_query)
    session.close()
    
    #print("Time of the 1st try to locate the data:{}".format(time.time() - start_time))
    
    for r in results:
        return r['title'], r['pageid'], r['id']
    
    #start_time = time.time()
    #2. If the href doesn't return an article, download this href
    #Try to get the article
    #If suceed, save the href and try to return the article and create a reference from the href
    art = wikipedia.get_article_by_href(href, lang)
    
    neo_query = "\n".join([
        'MERGE (u:Wikiurl{{url_lang:"{}"}})'.format(escp(href.unquoted) + "_" + lang),
        'ON CREATE SET u.url = "{}", u.lang = "{}", u.articleId = {}'.format(escp(href.unquoted), lang, art.page_id()),
        'WITH u MATCH (a:WikiArticle{{pageId:{}}})'.format(art.page_id()),
        'CREATE UNIQUE (u)-[:RedirectsTo]->(a)',
        'RETURN a.title as title, a.pageId as pageid, id(a) as id'
    ])
    
    session = w4n.driver.session()
    results = session.run(neo_query)
    session.close()
    #print("Time of the 2nd try to locate the data:{}".format(time.time() - start_time))
    
    for r in results:
        return r['title'], r['pageid'], r['id']
    
    #start_time = time.time()
    #3. If nothing has been returned till now, so the article doesn't exists, lets register it
    links = wikilinks.get_article_links_score(art)
    new_art_id = w4n.save_article(art, links, href, lang)
    #print("Time of the 3rd try to locate the data:{}".format(time.time() - start_time))
    
    return art.title(), art.page_id(), new_art_id

In [109]:
search_langs = [
    'de',
    'en',
    'fr',
    'es',
    'it',
    'nl',
    'ja',
    'pl',
    'ru',
    'ceb',
    'sv',
    'vi',
    'war'
]

In [110]:
#from urllib.parse import quote
import time

from ThreadPool import ThreadPool

def download_and_save_global_articles(href_en):
    start_time = time.time()
    
    #global_articles = list()

    #href_en = "MQTT"
    
    #1. Download english article and get links scores
    #art_en = wikipedia.get_article_by_href(href_en, "en")
    #links_scores = wikilinks.get_article_links_score(art_en)
    #global_articles.append((art_en, links_scores, href_en, "en"))
    #global_article_title = art_en.title()
    #global_article_pageid = art_en.page_id()
    
    #1.1 Get or download english article data
    print("Getting english article data...")
    art_en_title, art_en_page_id, art_en_id = get_or_download_pagedata_by_href(href_en, "en")
    
    global_article_title = art_en_title
    global_article_pageid = art_en_page_id
    
    #2. Get global links by english article title
    print("Getting global links...")
    #global_links = wikipedia.get_article_langlinks(art_en.title())
    #Quote title for correct results for cases of "C++" etc
    global_links = wikipedia.get_article_langlinks(UnquotedURL(art_en_title)) 
    

    #3. Download articles and generate links scores fot the global links
    #for art_lang, art_title in global_links:
        #art_href = quote(art_title)
        #art = wikipedia.get_article_by_href(art_href, art_lang)
        #art_links_scores = wikilinks.get_article_links_score(art)
        #global_articles.append((art, art_links_scores, art_href, art_lang))
        
    pool = ThreadPool(10)
        
    global_pageids = list()
    global_pageids.append(art_en_id)
    print("Getting global links articles data...") 
    #3.1 Register hrefs from the global links and get their register ids
    
    def _get_page_data(i, total, art_title, art_href, art_lang):
    
        print("{} {} - {}/{}".format(art_title, art_lang, i+1, total))
        #try:
        art_title, art_page_id, art_id = get_or_download_pagedata_by_href(art_href, art_lang)
        global_pageids.append(art_id)
        #except:
            #print("ERROR. Skiping.")
            
        print("Done {} {} - {}/{}".format(art_title, art_lang, i+1, total))
    
    
    for i, (art_lang, art_title, art_href) in enumerate(global_links):
        #if art_lang not in search_langs:
            #continue
            
        #art_href = quote(art_title) #get href by quoting title
        
        pool.add_task(_get_page_data, i+1, len(global_links), art_title, art_href, art_lang)
        
        #print("{} {} - {}/{}".format(art_title, art_lang, i+1, len(global_links)))
        #try:
            #art_title, art_page_id, art_id = get_or_download_pagedata_by_href(art_href, art_lang)
            #global_pageids.append(art_id)
        #xcept:
            #print("ERROR. Skiping.")
        
        
    pool.wait_completion()
    
    print(global_pageids)

    #4. Save them into the database and get their ids
    #global_pageids = list()
    #for art in global_articles:
        #new_page_ids = w4n.save_article(art[0], art[1], art[2], art[3])
        #global_pageids.append(new_page_ids)
    
    
    #5. Points every article to the global article of its kind
    print("Saving global articles...")
    neo_query = "\n".join([
        'MERGE (art:GlobalArticle{{title:"{}", en_page_id:{}}})'.format(global_article_title, global_article_pageid),
        "WITH art MATCH (target_art:WikiArticle) WHERE ID(target_art) IN {}".format(global_pageids),
        "CREATE UNIQUE (art)<-[:DerivesFrom]-(target_art)"
    ])    

    session = w4n.driver.session()
    session.run(neo_query)
    session.close()
    print("Done ({} seconds.)".format(time.time() - start_time))

In [111]:
from ThreadPool import ThreadPool
from IPython.display import clear_output
from collections import defaultdict
#from urllib.parse import unquote

def download_and_create_langlinks_dict(href_list):
    
    langlinks_dict = defaultdict(dict)
    done = [0]
    total = len(href_list)
    
    def _download_and_save_langlinks(href_en):
        for _lang, _title, _href in wikipedia.get_article_langlinks(href_en):
           langlinks_dict[_lang][_href.unquoted] = href_en.unquoted
        done[0] += 1
        clear_output(wait=True)
        print("Done {}/{}".format(done[0], total))
        
    pool = ThreadPool(10)
    
    for href in href_list:
        pool.add_task(_download_and_save_langlinks, href)
        
    pool.wait_completion()
    
    return langlinks_dict

In [112]:
#download_and_create_langlinks_dict(['C%2b%2b'])

In [113]:
download_and_save_global_articles(UnquotedURL("Generative adversarial networks"))

Getting english article data...
Getting global links...
Getting global links articles data...
生成对抗网络 zh - 3/2Generative adversarial networks simple - 2/2

Done Generative adversarial networks simple - 2/2
Done 生成对抗网络 zh - 3/2
[17458, 17477, 17474]
Saving global articles...
Done (4.829276084899902 seconds.)


In [119]:
query_title = "JavaScript"

query = "\n".join(['MATCH (n:GlobalArticle)<-[:DerivesFrom]-(:WikiArticle)-[l:LinksTo]->(u:Wikiurl)',
                   'WHERE n.title = "{}" RETURN l.score as score,u.url as url, u.lang as lang'
                  ]).format(query_title)

session = w4n.driver.session()
result = session.run(query)
session.close()


#Get results and fil the lang links list
from collections import defaultdict, Counter
lang_links = defaultdict(list)
for r in result:
    lang_links[r['lang']].append([r['url'], r['score']])
    
#Normalize scores for every lang
for lang in lang_links.keys():
    score_sum = 0
    for url, score in lang_links[lang]:
        score_sum += score
    for i,(url, score) in enumerate(lang_links[lang]):
        lang_links[lang][i].append(score/score_sum)
             
#Create lang links dict
href_list = list()
for href_en, _, _ in lang_links['en']:
    href_list.append(UnquotedURL(href_en))
ll_dict = download_and_create_langlinks_dict(href_list)

Done 321/321
Done 321/321


In [120]:
#Accumulate everything (assuming urls share href among languages)
#Maybe weight things by the number urls they have
#And get only things that the english version links to

#result_score = Counter()
all_langlinks = Counter()
all_langlinks_norm = Counter()
filtered_langlinks = Counter()
filtered_langlinks_norm = Counter()
only_english = Counter()

search_langs
for lang in lang_links.keys():
    for url, score, score_norm in lang_links[lang]:
        
        if lang == "en":
            global_url = url
            only_english[global_url] += score
        else:
            try:
                global_url = ll_dict[lang][url]
            except:
                continue
        
        if lang in search_langs:
            filtered_langlinks[global_url] += score
            filtered_langlinks_norm[global_url] += score_norm
        
        all_langlinks[global_url] += score
        all_langlinks_norm[global_url] += score_norm
            
        #result_score[global_url] += score_norm        

#result_score.most_common()

In [121]:
from tabulate import tabulate

n_elements = 20

list1 = all_langlinks.most_common(n_elements)#sorted(result_score.items(), key=lambda a: a[1], reverse=True)[:n_elements]
list2 = all_langlinks_norm.most_common(n_elements)#sorted(simple_link.items(), key=lambda a: a[1], reverse=True)[:n_elements]
list3 = filtered_langlinks.most_common(n_elements)#sorted(abstract_links.items(), key=lambda a: a[1], reverse=True)[:n_elements]
list4 = filtered_langlinks_norm.most_common(n_elements)#sorted(abstract_links.items(), key=lambda a: a[1], reverse=True)[:n_elements]
list5 = only_english.most_common(n_elements)#sorted(abstract_links.items(), key=lambda a: a[1], reverse=True)[:n_elements]

joined_list = list(zip(*list1)) + list(zip(*list2)) + list(zip(*list3)) + list(zip(*list4)) + list(zip(*list5))

joined_list = list(zip(*joined_list))

headers = [
    'All langlinks','Score',
    'All langlinks_norm','Score',
    'Filtered langlinks','Score',
    'Filtered langlinks_norm','Score',
    'Only english','Score'
]

folder_name = "benchmark langlinks scores"
with open(folder_name +"/" + query_title + ".txt", "w") as f:
    f.write(tabulate(joined_list, headers=headers))